<a href="https://colab.research.google.com/github/abay-qkt/madb-exploration/blob/main/%E3%83%90%E3%82%AF%E3%83%9E%E3%83%B3%E3%80%82%E3%81%AE%E6%8E%B2%E8%BC%89%E9%A0%86.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

- メディア芸術データベース（MADB）という、漫画やアニメなどに関する情報のデータベースあります。
- このデータを使った教材などいくつか紹介されていましたが、環境構築が必要だったり、一度データをローカルにダウンロードしたりする必要がありました。

- 2024年の１月からSPARQLという言語で直接データ引っ張ってくることができるようになりました。これで環境構築の手間なしにColab等でデータに触ってみることができます！

<メディア芸術データベースに関するリンク>
- [メディア芸術データベースについて](https://mediaarts-db.artmuseums.go.jp/about)
- [データセットおよびSPARQLクエリサービスの利用方法](https://mediag.bunka.go.jp/madb_lab/lod/howto/)
- [クエリサービス](https://mediag.bunka.go.jp/madb_lab/lod/sparql/?q=3)
- [MADBメタデータスキーマ仕様書.pdf](https://github.com/mediaarts-db/dataset/blob/main/doc/MADBメタデータスキーマ仕様書.pdf)



<SPARQLの参考になる記事>
- [SPARQL入門](https://www.aise.ics.saitama-u.ac.jp/~gotoh/IntroSPARQL.html)
- [ジャパンサーチのSPARQLエンドポイント](https://jpsearch.go.jp/rdf/sparql-explain/)

# はじめに

- 私は、「バクマン。」という漫画作品が好きです。週刊少年ジャンプで人気作家となりアニメ化を目指すというストーリーの作品です。主人公たちはアンケートでの順位をシビアに考えていて常に上位を目指しています。
- 物語の序盤で、「ジャンプ」のマンガ掲載順に関して以下の話がありました(あくまで作中での話ですが)。
```
全くの人気順ではないが ギャグマンガを除けば まあ 人気マンガが前の方 人気ないと後ろの方になるといってもいいくらいではあるな
```
(バクマン。3巻151ページ)

- 単行本派の私にとっては、各作品の掲載順がどうなっているのか全く分からずイメージが湧かなかったのですが、今回このデータベースを使って、人気作品の順位がどのような推移か、またバクマン。の掲載順位はどんな感じだったのかを調べてみました。

In [1]:
!pip install sparqlwrapper

In [2]:
import pandas as pd
import numpy as np
import plotly.express as px

In [3]:
from SPARQLWrapper import SPARQLWrapper

# エンドポイントの設定
sparql = SPARQLWrapper(endpoint='https://mediaarts-db.artmuseums.go.jp/sparql', returnFormat='json')

# クエリを投げてDataFrameを返す関数
def read_sparql(query):
  sparql.setQuery(query)
  response = sparql.queryAndConvert()  # 辞書形式で結果を受け取る
  df = pd.DataFrame(response["results"]["bindings"]).applymap(lambda x:x["value"])  # DataFrameに変換する
  df = df[response["head"]["vars"]]  # 列の並び替え
  return df

# データロード

In [4]:
query = """
PREFIX rdfs:   <http://www.w3.org/2000/01/rdf-schema#>
PREFIX schema: <https://schema.org/>
PREFIX class:  <https://mediaarts-db.artmuseums.go.jp/data/class#>

SELECT
    ?発表年月日
    ?ラベル
    ?タイトル
    ?サブタイトル
    ?開始ページ
    ?終了ページ
WHERE {
    ?マンガ雑誌単号 schema:isPartOf <https://mediaarts-db.artmuseums.go.jp/id/C119459> ;
    schema:genre "マンガ雑誌単号" ;
    rdfs:label ?ラベル ;
    schema:datePublished ?発表年月日 ;
    schema:hasPart [
        schema:genre "マンガ作品" ;
        schema:name ?タイトル ;
        schema:alternativeHeadline ?サブタイトル ;
        schema:pageStart ?開始ページ ;
        schema:pageEnd ?終了ページ ;
    ].
}
ORDER BY ?発表年月日 ?開始ページ
"""
df = read_sparql(query)

<ipython-input-3-fa3f9bac15c6>:10: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df = pd.DataFrame(response["results"]["bindings"]).applymap(lambda x:x["value"])  # DataFrameに変換する


In [5]:
df.head(10)

,発表年月日,ラベル,タイトル,サブタイトル,開始ページ,終了ページ
0,1969-11-03,週刊少年ジャンプ 1969.0 20,赤塚ギャグ笑待席,ゲバゲバ兄弟,103.0,117.0
1,1969-11-03,週刊少年ジャンプ 1969.0 20,鹿兵のケラケラ日記,スポーツの秋 ／ 味覚の秋 ／ ラムダロケットまたしっぱい ／ あみものの季節,118.0,119.0
2,1969-11-03,週刊少年ジャンプ 1969.0 20,デロリンマン,救世主誕生の巻,120.0,134.0
3,1969-11-03,週刊少年ジャンプ 1969.0 20,黄金仮面,くずれる顔,135.0,165.0
4,1969-11-03,週刊少年ジャンプ 1969.0 20,挑戦者ケーン,おれは挑戦する!の巻①,248.0,262.0
5,1969-11-03,週刊少年ジャンプ 1969.0 20,父の魂,はばたく隼人の巻,263.0,290.0
6,1969-11-03,週刊少年ジャンプ 1969.0 20,モサ,盗まれた金の巻,39.0,53.0
7,1969-11-03,週刊少年ジャンプ 1969.0 20,まんがコント55号,ゆうかい魔の巻,54.0,55.0
8,1969-11-03,週刊少年ジャンプ 1969.0 20,どうどう野郎,転校生の巻,56.0,70.0
9,1969-11-03,週刊少年ジャンプ 1969.0 20,ハレンチ学園,赤い嵐の巻,7.0,37.0


「ORDER BY ?発表年月日 ?開始ページ」していますが、開始ページの並び順が数値ではなく文字列順のソートになっています

In [6]:
df.dtypes

,0
発表年月日,object
ラベル,object
タイトル,object
サブタイトル,object
開始ページ,object
終了ページ,object


いったんすべてobject型で読み込んでいます

# 前処理

In [7]:
# 全部文字列になっているので、適切な型に変換する
df["発表年月日"]=pd.to_datetime(df["発表年月日"])
df["開始ページ"]=df["開始ページ"].astype(float).astype(int)
df["終了ページ"]=df["終了ページ"].astype(float).astype(int)

# 適切な型にしたので、改めてソート
df = df.sort_values(["発表年月日","開始ページ"]).reset_index(drop=True)

In [8]:
df["num_of_page"] = df["終了ページ"]-df["開始ページ"] # 各タイトルのページ数
df = df[df["num_of_page"]>10].reset_index(drop=True)  # 本編以外のものが混ざってそうだったので、ざっくりページ数で判断して除外
df = df.drop_duplicates(subset=["発表年月日","開始ページ"],keep='first').reset_index(drop=True) # 複数同時掲載の場合最初の1本だけ分析対象にする(今回順位を見たいので)

In [9]:
df["掲載順位"] = df.groupby(["発表年月日","ラベル"])["開始ページ"].rank().astype(int)  # 掲載順位
df["掲載作品数"] = df.groupby(["発表年月日","ラベル"])["タイトル"].transform("count")  # 掲載作品数
df["スコア"] = (df["掲載作品数"]-df["掲載順位"]+1) / df["掲載作品数"]  # 1位が1、最下位が0になるよう正規化した得点を付与

In [10]:
df

,発表年月日,ラベル,タイトル,サブタイトル,開始ページ,終了ページ,num_of_page,掲載順位,掲載作品数,スコア
0,1969-11-03,週刊少年ジャンプ 1969.0 20,ハレンチ学園,赤い嵐の巻,7,37,30,1,9,1.000000
1,1969-11-03,週刊少年ジャンプ 1969.0 20,モサ,盗まれた金の巻,39,53,14,2,9,0.888889
2,1969-11-03,週刊少年ジャンプ 1969.0 20,どうどう野郎,転校生の巻,56,70,14,3,9,0.777778
3,1969-11-03,週刊少年ジャンプ 1969.0 20,男一匹ガキ大将,万吉仁王立ちの巻,71,101,30,4,9,0.666667
4,1969-11-03,週刊少年ジャンプ 1969.0 20,赤塚ギャグ笑待席,ゲバゲバ兄弟,103,117,14,5,9,0.555556
...,...,...,...,...,...,...,...,...,...,...
39131,2017-07-31,週刊少年ジャンプ 2017.0 33,ぼくたちは勉強ができない,問23. 天才たちの花園に[x]は不可欠である,367,385,18,14,18,0.277778
39132,2017-07-31,週刊少年ジャンプ 2017.0 33,火ノ丸相撲,第153番 未来,387,405,18,15,18,0.222222
39133,2017-07-31,週刊少年ジャンプ 2017.0 33,青春兵器ナンバーワン,mission 37: ROMANCE DAWN,407,421,14,16,18,0.166667
39134,2017-07-31,週刊少年ジャンプ 2017.0 33,HUNTER×HUNTER,No.364 思惑,423,441,18,17,18,0.111111


# 分析

## 各作品の掲載順推移

### 可視化関数定義

In [11]:
# 指定したタイトルの順位の時系列プロットを可視化する関数
def plot_rank(df,titles):
  fig = px.line(df[df["タイトル"].isin(titles)],
                x='発表年月日',y="掲載順位",color='タイトル',
                hover_data=["タイトル","サブタイトル"])
  fig.update_layout(hovermode="x")
  fig.update_yaxes(autorange='reversed')
  fig.update_traces(
          hovertemplate ="<b>%{y}位</b>：%{customdata[0]}<br>%{customdata[1]}",
          mode='lines+markers',
          marker=dict(line_width=1, size=10))
  fig.update_layout(
      xaxis=dict(
        tickformat="%m月\n%Y",
        rangeslider=dict(visible=True),
        type="date",
        rangeselector=dict(  # ズーム幅をボタンで切り替えられるようにする
          buttons=list([
            dict(count=1,label="1y",step="year",stepmode="backward"),
            dict(count=3,label="2y",step="year",stepmode="backward"),
            dict(count=3,label="3y",step="year",stepmode="backward"),
            dict(step="all")
          ])
        )
      ),
      yaxis=dict(
        fixedrange= True  # y軸のズーム禁止
      )
  )
  fig.show()

### 有名作品との比較

In [12]:
titles_famous = [
    "こちら葛飾区亀有公園前派出所",
    "DRAGON　BALL",
    "ONE PIECE",
    "NARUTO-ナルト-",
    "BLEACH",
    "バクマン。",
]
plot_rank(df,titles_famous)

- 有名作品と比較してみました。バクマン。は思っていたより掲載順位の変動が激しい印象です。
- アニメが始まった2010年10月付近から順位が安定しているように見えます

### 同時期に連載開始した作品との比較

In [13]:
# 2008年連載開始 & 100回以上登場する作品
titles_2008 = (
    df.groupby("タイトル")["発表年月日"]
    .agg(["first","count"])
    .query("first.dt.year==2008 and count>100").index
    .to_list()
)
plot_rank(df,titles_2008)

- バクマン。と同時期に連載開始した作品と比較しました。この中だと、トリコはメディア展開されていて人気があった印象です。
- 今回連載が100回以上ある作品に限定していますが、すべて表示してみると数か月で終了している作品もあり、かなり厳しい世界なのだと改めて思いました。

### 有名作品や同時期の作品との比較

In [14]:
fig = px.box(df[df["タイトル"].isin(titles_2008+titles_famous)],x='タイトル',y='掲載順位')
fig.update_yaxes(autorange='reversed')
fig.show()

- こう並べてみると、DRAGON BALL、ONE PIECE、NARUTOは異常なくらい常に上位維持していますね

## 看板作品

その年に10回以上掲載されている作品の中で、平均掲載順位が高かった上位5作品を看板作品としてみました。

In [15]:
rep_df = (
    df
    .groupby([pd.Grouper(key='発表年月日',freq='YE'),"タイトル"])["掲載順位"]
    .agg(["mean","count"])  # 年ごとに掲載順位の平均値と掲載数を計算
    .rename(columns={"mean":"平均掲載順位","count":"掲載数"})  # カラム名変更
    .query("掲載数>10")  # その年の掲載回数が10より大きい作品のみを抽出
    .reset_index()
    .sort_values(["発表年月日","平均掲載順位"],ignore_index=True) # ソート
)
# 平均掲載順位が高い順に順位を振る。同じ値の場合最初に掲載された方を優先して順位を高くする
rep_df["年間順位"] = rep_df.groupby(["発表年月日"])["平均掲載順位"].rank(method='first').astype(int)


rep_df["disp_str"] = rep_df.apply(lambda x:"{}({:.1f})".format(x["タイトル"],x["平均掲載順位"]),axis=1)

rep_df_piv = rep_df.pivot(index='発表年月日',columns='年間順位',values='disp_str')
rep_df_piv.iloc[:,:5]

年間順位,1,2,3,4,5
発表年月日,,,,,
1970-12-31,ハレンチ学園(2.0),男一匹ガキ大将(4.2),ど根性ガエル(4.4),あらし!三匹(4.7),アニマル球場(5.3)
1971-12-31,荒野の少年イサム(1.3),男一匹ガキ大将(2.7),ど根性ガエル(3.7),侍ジャイアンツ(4.2),トイレット博士(5.0)
1972-12-31,荒野の少年イサム(2.1),男一匹ガキ大将(2.5),ど根性ガエル(3.1),マジンガーZ(3.4),ハレンチ学園(4.4)
1973-12-31,荒野の少年イサム(2.6),大ぼら一代(2.7),マジンガーZ(3.2),ど根性ガエル(3.3),男一匹ガキ大将(3.5)
1974-12-31,炎の巨人(2.5),大ぼら一代(3.2),トイレット博士(3.4),プレイボール(3.6),包丁人味平(5.3)
1975-12-31,ゼロの白鷹(3.5),ドーベルマン刑事(4.4),トイレット博士(5.0),サーキットの狼(5.1),アストロ球団(5.2)
1976-12-31,サーキットの狼(3.3),ドーベルマン刑事(5.1),悪たれ巨人(5.3),四丁目の怪人くん(5.5),1・2のアッホ!!(6.0)
1977-12-31,サーキットの狼(2.9),JUMP民話劇場(3.2),こちら葛飾区亀有公園前派出所(4.9),朝太郎伝(5.8),すすめ!!パイレーツ(6.0)
1978-12-31,さわやか万太郎(3.6),すすめ!!パイレーツ(4.2),ホールインワン(4.3),サーキットの狼(5.7),ルーズ!ルーズ!!(6.2)


- ONE PIECEが上位なんだろうなとは思っていましたが、思っていた以上に常に上位に位置していました。ONE PIECE、DRAGON BALLは圧倒的ですね。他にも上位に来ている作品はどれも知っているか、名前は聞いたことのある作品が多い印象です。
- バクマン。も2011年に4位に来ています！

# 終わりに

- ジャンプ本誌は一度も購入したことがなかったので、バクマン。作中に出てくる、掲載順や打ち切りの話などのイメージがあまり湧かなかったのですが、ワンピース、NARUTOといった人気作品は常に上位に来ているところを見ると、人気作は最初の方に掲載している、というのは本当にそんな感じでした。
- なお、私はONE PIECEもNARUTOも好きですが、それ以上にバクマン。が好きです！